In [30]:
from platform import python_version

print(python_version())

3.7.5


In [70]:
import os
import numpy as np
from os import listdir
from PIL import Image
import time
import tensorflow as tf
from tensorflow.keras import layers,models,optimizers
from keras import backend as K
import matplotlib.pyplot as plt

In [71]:
path="datasets/ofg_family/"
randomiser = np.random.RandomState(123)

In [34]:
def generate_image(family_dir):
    dic={}
    sub=[a for a in listdir(path+"/"+family_dir)]
    
    for ele in sub:
        mypath = path+"/"+family_dir+"/"+ele+"/"
        onlyfiles = [mypath+f for f in listdir(mypath)]
        
        addr = randomiser.choice(onlyfiles)
        original_img = np.array(Image.open(addr).resize((64,64),Image.ANTIALIAS))
        if ele[0].lower()=='f':
            dic['father'] = original_img
        elif ele[0].lower()=='m':
            dic['mother'] = original_img
        elif ele.lower()=='child_male':
            dic['child'] = original_img    
            dic['gender']=np.zeros((original_img.shape))
        elif ele.lower()=='child_female':
            dic['child'] = original_img    
            dic['gender'] = np.ones((original_img.shape))
    return [dic['father'],dic['mother'],dic['gender'],dic['child']]

def generate_batch(families_batch):
    np_images=[]
    
    for family in families_batch:
        res = generate_image(family)
        if( res != None):
            np_images.append(res)
    
    return np_images

In [35]:
#full_data = load_npdata(path)
#print(len(full_data))

for r, d, f in os.walk(path):
        all_families=d
        break

randomiser.shuffle(all_families)

train_families = all_families[:-500]
test_families = all_families[-500:]

create encoder decoder supervised pipeling operation
input: stacked images of father and mother (32,32,6)
encoder -> decoder -> image reconstruction
reconstruction loss w/ target image as child

In [36]:
concat = tf.keras.layers.Concatenate()

family_data = generate_image(all_families[0])
inp = concat([family_data[0],family_data[1]])
child = family_data[3]
child = tf.convert_to_tensor(child, dtype=tf.float32)


In [37]:
OUTPUT_CHANNELS = 3

In [38]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02) 
  

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [39]:
def upsample(filters, size):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())


  result.add(tf.keras.layers.ReLU())

  return result

In [40]:
sd_random_normal_init = 0.135

In [41]:
def EncoderNN():
  down_stack = [
    downsample(32, 4, apply_batchnorm=True),
    downsample(64, 4, apply_batchnorm=True),
    downsample(128, 4, apply_batchnorm=True) 
  ]

  up_stack = [
    upsample(64, 4),
    upsample(32, 4),
    upsample(3, 4)
  ]

  initializer = tf.random_normal_initializer(0., sd_random_normal_init)
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 2,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='sigmoid')

  concat = tf.keras.layers.Concatenate()

  inputs = tf.keras.layers.Input(shape=[64,64,6])
  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = concat([x, skip])
    
  x = last(x)


  return tf.keras.Model(inputs=inputs, outputs=x)

In [42]:
"""
def custom_loss():
    def loss(y_true,y_pred):
        print(y_true.shape)
        print(y_pred.shape)
        return tf.reduce_mean(tf.abs(y_true - y_pred))
        #return tf.reduce_mean(tf.image.total_variation(tf.concat([y_true,y_pred],axis=0)))
    return loss
"""

'\ndef custom_loss():\n    def loss(y_true,y_pred):\n        print(y_true.shape)\n        print(y_pred.shape)\n        return tf.reduce_mean(tf.abs(y_true - y_pred))\n        #return tf.reduce_mean(tf.image.total_variation(tf.concat([y_true,y_pred],axis=0)))\n    return loss\n'

inp = concat([full_data[0][0],full_data[0][1]])
inp = tf.convert_to_tensor(inp, dtype=tf.float32)
X_train = tf.reshape(inp,([1,32,32,6]))
Y_train = tf.reshape(full_data[0][3],(1,32,32,3))

encoder.compile(loss=custom_loss(),
              optimizer=tf.keras.optimizers.RMSprop())
history = encoder.fit(X_train, Y_train,
                    batch_size=1,
                    epochs=1)

gen_output = encoder(inp[tf.newaxis,...], training=False)

In [43]:
concat = tf.keras.layers.Concatenate()

In [44]:
# encoder = EncoderNN()
# encoder.compile(loss=custom_loss(),optimizer=tf.keras.optimizers.RMSprop())

In [45]:
"""
epochs = 20
batch = 375

for epoch in range(epochs):
    print("Epoch ", epoch , " .....")
    for i in range(len(train_families)//batch):
        batch_data = np.asarray(generate_batch(train_families[i*batch:(i+1)*batch]))
        
        print("Generated batch", batch_data.shape)
        
        X_train = tf.convert_to_tensor(concat([batch_data[:,0],batch_data[:,1]]),dtype =tf.float32)
        print("Batch converted to tensor")
        
        Y_train = batch_data[:,3]
        history = encoder.fit(X_train, Y_train, batch_size=batch)
print("Training DONE!")
"""

'\nepochs = 20\nbatch = 375\n\nfor epoch in range(epochs):\n    print("Epoch ", epoch , " .....")\n    for i in range(len(train_families)//batch):\n        batch_data = np.asarray(generate_batch(train_families[i*batch:(i+1)*batch]))\n        \n        print("Generated batch", batch_data.shape)\n        \n        X_train = tf.convert_to_tensor(concat([batch_data[:,0],batch_data[:,1]]),dtype =tf.float32)\n        print("Batch converted to tensor")\n        \n        Y_train = batch_data[:,3]\n        history = encoder.fit(X_train, Y_train, batch_size=batch)\nprint("Training DONE!")\n'

In [46]:
"""
family_data = generate_image(all_families[700])
inp = concat([family_data[0],family_data[1]])
inp = tf.cast(inp, tf.float32)
gen_output = encoder(inp[tf.newaxis,...], training=False)
print(tf.reduce_min(gen_output))
print(tf.reduce_max(gen_output))
plt.imshow(gen_output[0,...])
"""

'\nfamily_data = generate_image(all_families[700])\ninp = concat([family_data[0],family_data[1]])\ninp = tf.cast(inp, tf.float32)\ngen_output = encoder(inp[tf.newaxis,...], training=False)\nprint(tf.reduce_min(gen_output))\nprint(tf.reduce_max(gen_output))\nplt.imshow(gen_output[0,...])\n'

In [47]:
def Discriminator():
    initializer = tf.random_normal_initializer(0., sd_random_normal_init)

    inp = tf.keras.layers.Input(shape=[64, 64, 6], name='input_image')
    tar = tf.keras.layers.Input(shape=[64, 64, 3], name='target_image')

    x = tf.reshape(tf.keras.layers.concatenate([inp, tar]),(1,64,64,9))
    down1 = downsample(32, 4, True)(x)
    down2 = downsample(64, 4)(down1) 
    down3 = downsample(128, 4)(down2) 

    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) 
    conv = tf.keras.layers.Conv2D(256, 3, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1)

    batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

    leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)

    last = tf.keras.layers.Conv2D(1, 3, strides=1,
                                kernel_initializer=initializer)(zero_pad2) 
    return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [48]:
"""
gen_output = encoder(inp[tf.newaxis,...], training=False)

target = family_data[2]
discriminator = Discriminator()
disc_out = discriminator([inp, tf.squeeze(gen_output)], training=False)
plt.imshow(disc_out[0,...,-1], vmin=-20, vmax=20, cmap='RdBu_r')
plt.colorbar()
"""

"\ngen_output = encoder(inp[tf.newaxis,...], training=False)\n\ntarget = family_data[2]\ndiscriminator = Discriminator()\ndisc_out = discriminator([inp, tf.squeeze(gen_output)], training=False)\nplt.imshow(disc_out[0,...,-1], vmin=-20, vmax=20, cmap='RdBu_r')\nplt.colorbar()\n"

In [49]:
LAMBDA = 100

In [50]:
# loss_object = tf.keras.losses.BinaryCrossentropy()

In [92]:
def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = loss_object(tf.ones_like(disc_real_output,dtype=tf.float32), disc_real_output)
    generated_loss = loss_object(tf.zeros_like(disc_generated_output,dtype=tf.float32), disc_generated_output)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss

In [93]:
def generator_loss(disc_generated_output, gen_output, target):
    gan_loss = loss_object(tf.ones_like(disc_generated_output,dtype=tf.float32), disc_generated_output)
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
    total_gen_loss = gan_loss + (LAMBDA * l1_loss)
    return total_gen_loss

In [94]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [95]:
# checkpoint_dir = './checkpoint'
# checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
# checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
#                                  discriminator_optimizer=discriminator_optimizer,
#                                  generator=encoder,
#                                  discriminator=discriminator)

In [96]:
EPOCHS = 150

In [109]:
@tf.function
def train_step(inp_batch, target_batch,b_size):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        
        inp_batch = tf.unstack(inp_batch)
        target_batch = tf.unstack(target_batch)
        
        gen_loss =tf.constant(0,dtype='float32')
        disc_loss =tf.constant(0,dtype='float32')
        
        for idx,input_image in enumerate(inp_batch):
                
            print(idx)
            
            gen_output = encoder(input_image[tf.newaxis,...], training=True)
            disc_real_output = discriminator([input_image, target_batch[idx]], training=True)
            disc_generated_output = discriminator([input_image, tf.squeeze(gen_output)], training=True)
            print(loss_object(tf.ones_like(disc_real_output,dtype=tf.float32), disc_real_output).numpy())
            gen_loss = gen_loss 
#             + generator_loss(disc_generated_output, gen_output, target_batch[idx])
#             print(tensor_to_array(gen_loss))
            disc_loss = disc_loss + discriminator_loss(disc_real_output, disc_generated_output)
    
    
    gen_loss/=b_size
    disc_loss/=b_size
    tf.print("GEN_LOSS:",gen_loss)
    tf.print("DISC_LOSS",disc_loss)
    
    generator_gradients = gen_tape.gradient(gen_loss,
                                          encoder.trainable_variables)
    discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(generator_gradients,
                                          encoder.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))


In [110]:
def fit(train_ds, epochs, test_ds,batch):
    for epoch in range(epochs):
        start = time.time()
        for i in range(len(train_ds)//batch):
            batch_data = np.asarray(generate_batch(train_ds[i*batch:(i+1)*batch]))

            print("Generated batch", batch_data.shape)

            X_train = tf.convert_to_tensor(concat([batch_data[:,0],batch_data[:,1]]),dtype =tf.float32)
            print("Xtrain",X_train.shape)
            print("Batch converted to tensor")

            Y_train = tf.convert_to_tensor(batch_data[:,3],dtype =tf.float32)
            train_step(X_train,Y_train,batch)

In [111]:
tf.executing_eagerly()
tf.compat.v1.enable_eager_execution()

In [112]:
train_dataset = all_families[:-500]
test_dataset = all_families[-500:]
EPOCHS = 1
batch = 10
encoder = EncoderNN()
discriminator = Discriminator()
fit(train_dataset, EPOCHS, test_dataset,batch)


Generated batch (10, 4, 64, 64, 3)
Xtrain (10, 64, 64, 6)
Batch converted to tensor
0


AttributeError: in converted code:

    <ipython-input-109-cc945ef1f909>:18 train_step  *
        print(loss_object(tf.ones_like(disc_real_output,dtype=tf.float32), disc_real_output).numpy())

    AttributeError: 'Tensor' object has no attribute 'numpy'


In [ ]:
train_dataset = all_families[:-500]
test_dataset = all_families[-500:]
EPOCHS = 1

In [ ]:
fit(train_dataset, EPOCHS, test_dataset)

In [ ]:
with tf.Session() as sess:
    print()